# Solving problems by Searching

This notebook serves as supporting material for topics covered in **Chapter 3 - Solving Problems by Searching** and **Chapter 4 - Beyond Classical Search** from the book *Artificial Intelligence: A Modern Approach.* This notebook uses implementations from [search.py](https://github.com/aimacode/aima-python/blob/master/search.py) module. Let's start by importing everything from search module.

In [6]:

import random
import bisect
# Needed to hide warnings in the matplotlib sections
import warnings
#from numpy import *
warnings.filterwarnings("ignore")

For visualisations, we use networkx and matplotlib to show the map in the notebook and we use ipywidgets to interact with the map to see how the searching algorithm works. These are imported as required in `notebook.py`.

## CONTENTS

* Genetic Algorithm


## GENETIC ALGORITHM

Genetic algorithms (or GA) are inspired by natural evolution and are particularly useful in optimization and search problems with large state spaces.

Given a problem, algorithms in the domain make use of a *population* of solutions (also called *states*), where each solution/state represents a feasible solution. At each iteration (often called *generation*), the population gets updated using methods inspired by biology and evolution, like *crossover*, *mutation* and *natural selection*.

PERMUTATION ENCODING

GA Parameters:
We now need to define the maximum size of each population. Larger populations have more variation but are computationally more  expensive to run algorithms on.
As our population is not very large, we can afford to keep a relatively large mutation rate.
Termination after a predefined number of generations.
N is the size of the chromosmes, and [0,..,N-1] is the alphabet

In [12]:
max_population = 100
mutation_rate = 0.07 # 7% of the chromosones are mutated
ngen = 100 # maximum number of generations
N = 16 # chormosome size
gene_pool = list(range(1,N+1)) # alphabet
#gene_pool.remove(0)
print(gene_pool)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


Great! Now, we need to define the most important metric for the genetic algorithm, i.e the fitness function. This will simply return the number of matching characters between the generated sample and the target phrase.

In [13]:
def fitness_fn(sample):
    # initialize fitness to 0
    fitness = 0
    for i in range(len(sample)):
        # increment fitness by 1 for every matching character
        if sample[i] == target[i]:
           fitness += 1
    return fitness
# target
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

To generate `ngen` number of generations, we run a `for` loop `ngen` number of times. After each generation, we calculate the fitness of the best individual of the generation and compare it to the value of `f_thres` using the `fitness_threshold` function. After every generation, we print out the best individual of the generation and the corresponding fitness value. Lets now write a function to do this.

In [12]:
import time

def genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen=1200, pmut=0.1):
    for generation in range(int(ngen)):
        seconds = time.time()
        # Elitism may be here - ADDED
        previous_best = max(population, key=fitness_fn)
        population = [mutate2(order_crossover(*select(2, population, fitness_fn)), pmut) for i in range(len(population)-1)]
        population.append(previous_best)
        # stores the individual genome with the highest fitness in the current population
        current_best = max(population, key=fitness_fn)
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_fn(current_best)}\r', end='')
        fitness_best = fitness_fn(current_best)
        result_time = time.time()-seconds
        with open('genetic-data.csv','a') as f:
            f.write(f'{generation};{1/fitness_best};{result_time};\n')
        print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_best}\r')
    return max(population, key=fitness_fn)       

def init_population(pop_number, gene_pool):
    # a chromosome is a random permutation of the alphabet
    population = []
    for _ in range(pop_number):
        # Shuffle the gene pool and take the first pool_size elements as an individual
        v = gene_pool[:]
        random.shuffle(v)
        population.append(v)
    return population

def select(r, population, fitness_fn):
    fitnesses = map(fitness_fn, population)
    #scaling here
    sampler = weighted_sampler(population, fitnesses)
    return [sampler() for i in range(r)]

def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]
    # bisect(a,x) -> insertion position of a in a sorted list x - AL REVES

def uniform_crossover(x, y):
    # x, y permutations of the alphabet
    n = 0
    child = [-1] * N
    indexes = [0] * N
    # de x se copian los valores de las posiciones con indexex[i] == 1 en las mismas posiciones en child
    for i in  range(N):
        indexes[i] = random.randint(0,1) 
        if indexes[i] == 1:
            child[i] = x[i]
            n += 1
    # El resto (N-n) se copia de y en su orden relativo, desde el principio
    i = 0 # indice en y
    k = 0 # indice en child
    for t in range(N-n):
        while y[i] in child[:]:
            i += 1
        while child[k] != -1:
            k += 1
        child[k] = y[i]
        i += 1   
    return child
    

def order_crossover(parent1, parent2):
    r1 = random.randint(1, len(parent1) - 3)
    r2 = random.randint(r1, len(parent1) - 2)

    offspring = [-1] * r1 + parent1[r1:r2] + [-1] * (len(parent1) - r2)
    j = 0
    for i in parent2:
        if i not in offspring:
            offspring[(r2 + j) % len(parent2)] = i
            j += 1

    return offspring

def mutate2(x, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    i, j = random.sample(range(N), 2)
    x[i], x[j] = x[j], x[i]
    return x

The function defined above is essentially the same as the one defined in `search.py` with the added functionality of printing out the data of each generation.

We have defined all the required functions and variables. Let's now create a new population and test the function we wrote above.

In [18]:
population = init_population(max_population, gene_pool)
#print(population)
solution = genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen, mutation_rate)
print("Target: ")
print(target)
print("Solution: ")
print(solution)
print("Fitness: ")
print(fitness_fn(solution))

Current best: [14, 16, 3, 10, 1, 9, 6, 4, 7, 2, 11, 12, 13, 8, 15, 5]		Generation: 0		Fitness: 5
Current best: [5, 16, 10, 4, 6, 15, 7, 8, 14, 1, 11, 3, 13, 12, 9, 2]		Generation: 1		Fitness: 5
Current best: [1, 16, 15, 3, 5, 14, 7, 10, 9, 6, 11, 13, 12, 2, 4, 8]		Generation: 2		Fitness: 5
Current best: [1, 16, 4, 3, 15, 5, 7, 2, 9, 10, 11, 13, 12, 6, 14, 8]		Generation: 3		Fitness: 5
Current best: [12, 7, 3, 11, 2, 4, 1, 6, 9, 10, 5, 8, 13, 14, 15, 16]		Generation: 4		Fitness: 7
Current best: [12, 7, 3, 11, 2, 4, 1, 6, 9, 10, 5, 8, 13, 14, 15, 16]		Generation: 5		Fitness: 7
Current best: [1, 7, 3, 11, 4, 2, 12, 10, 9, 6, 5, 8, 13, 14, 15, 16]		Generation: 6		Fitness: 7
Current best: [1, 7, 3, 11, 4, 2, 12, 10, 9, 6, 5, 8, 13, 14, 15, 16]		Generation: 7		Fitness: 7
Current best: [12, 7, 3, 4, 5, 11, 9, 8, 13, 10, 2, 14, 1, 6, 15, 16]		Generation: 8		Fitness: 7
Current best: [1, 2, 3, 12, 5, 7, 10, 9, 6, 11, 8, 14, 13, 4, 15, 16]		Generation: 9		Fitness: 7
Current best: [1, 7, 3, 15, 4,

In [4]:
import copy
class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N = int(((8*len(lista)+1)**0.5)-1)/2
        # print(self.N)
        self.ciudades = list(range(int(self.N)))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]

    def residual(self, nodes):
        grafocopia = copy.deepcopy(self)
        
        grafocopia.arcos = []
        # Filtra las distancias para mantener solo las ciudades en 'nodes'
        new_dist = []
        for i in range(len(nodes)):
            row = []
            for j in nodes:
                row.append(grafocopia.Dist[nodes[i]][j])
            #Añadimos la lista de arcos, teniendo en cuenta la simetria de la matriz
            for j in range(i + 1, len(nodes)):
                grafocopia.arcos.append(grafocopia.Dist[i][j])
            new_dist.append(row)

        grafocopia.Dist=new_dist
        # Actualiza la lista de ciudades
        grafocopia.ciudades = nodes
        
        return grafocopia

        

In [7]:
prueba = [ 
    0,
    1, 0,
    6, 2, 0,
    4, 5, 3, 0,
]


gr21_commented_4 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    #450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    #170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    #240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    #380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

fri26_commented_5 = [
    0,
    83, 0,
    93, 40, 0,
    129, 53, 42, 0,
    133, 62, 42, 11, 0,
    139, 64, 49, 11, 9, 0,
    151, 91, 59, 46, 35, 39, 0,
    169, 116, 81, 72, 61, 65, 26, 0,
    135, 93, 54, 65, 55, 63, 34, 37, 0,
    114, 84, 44, 70, 62, 71, 52, 59, 22, 0,
    110, 95, 58, 88, 82, 90, 71, 75, 39, 20, 0,
    98, 98, 64, 100, 95, 103, 88, 92, 56, 36, 18, 0,
    99, 89, 54, 89, 84, 92, 77, 83, 47, 26, 11, 11, 0,
    95, 68, 31, 66, 62, 71, 63, 76, 40, 20, 27, 34, 23, 0,
    81, 67, 36, 76, 74, 82, 78, 91, 55, 34, 32, 31, 24, 15, 0,
    152, 127, 86, 102, 93, 100, 66, 54, 37, 43, 42, 56, 53, 62, 73, 0,
    159, 156, 117, 142, 133, 141, 110, 98, 78, 74, 61, 63, 68, 87, 92, 44, 0,
    181, 175, 135, 156, 146, 153, 119, 103, 91, 91, 80, 85, 89, 106, 112, 54, 22, 0,
    172, 152, 112, 127, 117, 124, 88, 70, 62, 68, 64, 75, 74, 87, 96, 26, 34, 33, 0,
    185, 165, 125, 139, 128, 135, 98, 78, 74, 82, 77, 87, 87, 100, 109, 39, 38, 29, 13, 0,
    147, 160, 124, 155, 148, 156, 130, 122, 96, 86, 68, 62, 71, 93, 93, 68, 30, 46, 63, 68, 0,
    #157, 180, 147, 180, 173, 181, 156, 148, 122, 111, 92, 83, 93, 116, 113, 94, 53, 64, 87, 90, 26, 0,
    #185, 223, 193, 228, 222, 230, 206, 198, 172, 160, 140, 129, 140, 163, 158, 144, 102, 107, 135, 136, 77, 50, 0,
    #220, 268, 241, 278, 272, 280, 257, 250, 223, 210, 190, 178, 189, 212, 205, 196, 154, 157, 186, 186, 128, 102, 51, 0,
    #127, 179, 157, 197, 194, 202, 188, 188, 155, 136, 116, 100, 111, 132, 122, 139, 109, 125, 141, 148, 80, 65, 64, 93, 0,
    #181, 197, 161, 190, 182, 190, 160, 148, 128, 121, 103, 99, 107, 130, 130, 95, 51, 51, 81, 79, 37, 27, 58, 107, 90, 0
]
N = 16 # chormosome size
gene_pool = list(range(1,N+1)) # alphabet
grafo = Grafo(gr17)
population = init_population(100, gene_pool)


In [8]:
def fitness(q):
    non_attacking = 0
    for row1 in range(len(q)):
        for row2 in range(row1+1, len(q)):
            col1 = int(q[row1])
            col2 = int(q[row2])
            row_diff = row1 - row2
            col_diff = col1 - col2

            if col1 != col2 and row_diff != col_diff and row_diff != -col_diff:
                non_attacking += 1

    return non_attacking
#Para comprobar con A* tenemos que hacer la inversa de la inversa del coste
#Coste(T) = coste(0,i1) + coste(i1,i2) + ... + coste(in-1,0)

def fitnessTSP(ind):
    coste =grafo.Dist[0][ind[0]]
    for i in range(1,len(ind)):
        coste+=grafo.Dist[ind[i-1]][ind[i]]
    coste+=grafo.Dist[ind[len(ind)-1]][0]
    return 1/coste

In [14]:
N = 16 # chormosome size
gene_pool = list(range(1,N+1)) # alphabet
grafo = Grafo(gr17)
population = init_population(100, gene_pool)
with open('genetic-data.csv','a') as f:
        f.write('gr17\n')
for ngens in [100,150,200,250,300]:
     with open('genetic-data.csv','a') as f:
        f.write(f'{ngens}\n')
     for _ in range(10):
        solution = genetic_algorithm_stepwise(population, fitnessTSP, gene_pool, ngen=ngens)

grafo = Grafo(gr21_commented_4)
with open('genetic-data.csv','a') as f:
        f.write('gr21_commented_4\n')
for ngens in [100,150,200,250,300]:
        with open('genetic-data.csv','a') as f:
             f.write(f'{ngens}\n')
        for _ in range(10):
                solution = genetic_algorithm_stepwise(population, fitnessTSP, gene_pool, ngen=ngens)

N = 20# chormosome size
gene_pool = list(range(1,N+1)) # alphabet
population = init_population(100, gene_pool)
grafo = Grafo(gr21)
with open('genetic-data.csv','a') as f:
        f.write('gr21\n')
for ngens in [100,150,200,250,300]:
     with open('genetic-data.csv','a') as f:
        f.write(f'{ngens}\n')
     for _ in range(10):
        solution = genetic_algorithm_stepwise(population, fitnessTSP, gene_pool, ngen=ngens)

grafo = Grafo(fri26_commented_5) 
with open('genetic-data.csv','a') as f:
        f.write('fri26_commented_5\n')
for ngens in [100,150,200,250,300]:
     with open('genetic-data.csv','a') as f:
        f.write(f'{ngens}\n')
     for _ in range(10):
        solution = genetic_algorithm_stepwise(population, fitnessTSP, gene_pool, ngen=ngens)
#print(solution)
#print(fitness(solution))

Current best: [14, 10, 4, 7, 6, 13, 2, 9, 1, 15, 12, 11, 8, 5, 16, 3]		Generation: 0		Fitness: 0.0002907822041291073
Current best: [16, 14, 6, 3, 8, 15, 7, 2, 1, 9, 10, 4, 5, 13, 12, 11]		Generation: 1		Fitness: 0.0002918004085205719
Current best: [12, 11, 8, 5, 16, 3, 7, 2, 1, 9, 10, 4, 14, 6, 13, 15]		Generation: 2		Fitness: 0.00031416902293433867
Current best: [12, 11, 8, 5, 16, 3, 7, 2, 1, 9, 10, 4, 14, 6, 13, 15]		Generation: 3		Fitness: 0.00031416902293433867
Current best: [8, 5, 6, 12, 16, 3, 7, 2, 1, 9, 10, 4, 14, 13, 15, 11]		Generation: 4		Fitness: 0.0003238341968911917
Current best: [8, 5, 6, 12, 16, 3, 7, 2, 1, 9, 10, 4, 14, 13, 15, 11]		Generation: 5		Fitness: 0.0003238341968911917
Current best: [8, 5, 6, 12, 16, 3, 7, 2, 1, 9, 10, 4, 14, 13, 15, 11]		Generation: 6		Fitness: 0.0003238341968911917
Current best: [12, 16, 3, 7, 2, 1, 9, 10, 4, 14, 13, 15, 11, 8, 5, 6]		Generation: 7		Fitness: 0.0003699593044765076
Current best: [12, 16, 3, 7, 2, 1, 9, 10, 4, 14, 13, 15, 11, 8

KeyboardInterrupt: 

This is where we conclude Genetic Algorithms.

<br>
This concludes the notebook.
Hope you learned something new!